In [2]:
import numpy as np
import pandas as pd

c:\Users\benka\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Train data

In [3]:
df_train = pd.read_csv('train.csv')
df_train.head()

,book_id,user_id,rating
0,7260,20145,3.5
1,243238,85182,4.0
2,9135,45973,1.0
3,18671,63554,3.0
4,243293,81002,5.0


In [4]:
print(df_train.isnull().sum())

book_id    0
user_id    0
rating     0
dtype: int64


In [5]:
n_users = df_train.user_id.nunique()
n_items = df_train.book_id.nunique()
print ('Number of users = ' + str(n_users) + ' | Number of books = ' + str(n_items))

Number of users = 18905 | Number of books = 15712


In [6]:
df_train.describe()

,book_id,user_id,rating
count,100523.000000,100523.000000,100523.000000
mean,49799.616217,45473.754852,2.361738
std,82711.354018,26921.563600,1.106863
min,1.000000,37.000000,1.000000
25%,3754.000000,22696.000000,1.500000
50%,11916.000000,44711.000000,2.000000
75%,37735.000000,68147.000000,3.000000
max,249242.000000,94399.000000,5.000000


index not starting at zero -> id much higher than nb of line/columns 

### Test data

In [7]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,book_id,user_id
0,0,3786,40484
1,1,1985,47039
2,2,2290,60111
3,3,118657,64447
4,4,1560,2953


In [8]:
print(df_test.isnull().sum())

id         0
book_id    0
user_id    0
dtype: int64


In [9]:
n_users_test = df_test.user_id.unique().shape[0]
n_items_test = df_test.book_id.unique().shape[0]
print ('Number of users = ' + str(n_users_test) + ' | Number of books = ' + str(n_items_test))

Number of users = 6519 | Number of books = 9371


### Recommender

In [10]:
def map_ids(data, column_name):
    """Map data IDs to zero-based indices"""
    unique_ids = data[column_name].unique()
    id_to_index = {old: new for new, old in enumerate(unique_ids)}
    return data[column_name].map(id_to_index), id_to_index

# Applying the mapping function to both train and test data
df_train['user_id'], user_mapping = map_ids(df_train, 'user_id')
df_train['book_id'], book_mapping = map_ids(df_train, 'book_id')
#test_data['user_id'] = test_data['user_id'].map(user_mapping)
#test_data['book_id'] = test_data['book_id'].map(book_mapping)

n_users = df_train['user_id'].nunique()
n_books = df_train['book_id'].nunique()

In [11]:
df_train.head()

,book_id,user_id,rating
0,0,0,3.5
1,1,1,4.0
2,2,2,1.0
3,3,3,3.0
4,4,4,5.0


In [12]:
df_train.describe()

,book_id,user_id,rating
count,100523.000000,100523.000000,100523.000000
mean,4985.189409,4149.397730,2.361738
std,4030.433657,4360.990111,1.106863
min,0.000000,0.000000,1.000000
25%,1536.000000,919.000000,1.500000
50%,3951.000000,2559.000000,2.000000
75%,7761.000000,5870.000000,3.000000
max,15711.000000,18904.000000,5.000000


now value of id max is same as number of line/columns

In [13]:
def create_data_matrix(data, n_users, n_books):
    """Create user-item matrix"""
    data_matrix = np.zeros((n_users, n_books))
    for line in data.itertuples():
        data_matrix[line[2], line[1]] = line[3]
    return data_matrix

train_data_matrix = create_data_matrix(df_train, n_users, n_books)
#test_data_matrix = create_data_matrix(test_data, n_users, n_books)


In [14]:
print(train_data_matrix.shape)

(18905, 15712)


In [15]:
train_data_matrix

array([[3.5, 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 4. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 1. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [16]:
## check for one value 
df_train[df_train["user_id"]==0]

,book_id,user_id,rating
0,0,0,3.5
1412,1259,0,2.5
11445,2989,0,2.0
19016,5684,0,1.5
22810,2641,0,2.5
46391,2482,0,2.0
52041,2389,0,2.5
54017,1556,0,2.0
59832,2742,0,3.5
61883,103,0,2.0


In [17]:
train_data_matrix[0][2989]

2.0

### Item-based similarity

Item-based better because each user will likely rate only a small subset of the available books --> user-item matrix will be sparse

In [18]:
from sklearn.metrics.pairwise import pairwise_distances
item_similarity = 1 - pairwise_distances(train_data_matrix.T, metric='cosine')

In [18]:
def item_based_predict(ratings, similarity):
    filled_matrix = np.zeros((n_users, n_items))
    # loop over all the users
    for u in range(n_users):
        # get the items rated by this user
        ranked_items_indices = train_data_matrix[u,:].nonzero()[0]
        for i in range(n_items):
            numerator = 0
            denominator = 0
            for j in ranked_items_indices:
                numerator+=item_similarity[i,j]*train_data_matrix[u,j]
                denominator+=np.abs(item_similarity[i,j])
            if denominator>0:
                filled_matrix[u,i]= numerator/denominator
            else:
                # simply take a random rating in that case 
                filled_matrix[u,i]= np.random.randint(1,6)
    return filled_matrix        

item_prediction = item_based_predict(train_data_matrix, item_similarity)
print(item_prediction)

KeyboardInterrupt: 

Try something faster because loops take too much time: 

In [19]:
def item_based_predict(ratings, similarity):
    # Calculate the dot product between the ratings and the similarity matrix
    weighted_sum = ratings.dot(similarity)
    
    # Calculate the sum of similarities for each user and item pair
    sum_of_similarities = np.abs(similarity).dot((ratings > 0).astype(int).T).T
    
    # Prevent division by zero
    sum_of_similarities[sum_of_similarities == 0] = 1
    
    # Calculate the predicted ratings
    predictions = weighted_sum / sum_of_similarities
    
    return predictions

item_prediction = item_based_predict(train_data_matrix, item_similarity)
print(item_prediction)

[[2.69590358 0.         2.         ... 0.         0.         0.        ]
 [0.         4.         0.         ... 0.         0.         0.        ]
 [1.56190587 0.         1.82738198 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
